In [2]:
import torch

from transformers.models.mamba import MambaForCausalLM as MambaForCausalLM_orig, MambaConfig as MambaConfig_orig

from src.transformers.models.mamba import MambaForCausalLM, MambaConfig

In [3]:
torch.manual_seed(34567)
config = MambaConfig_orig(vocab_size=60, hidden_size=64, num_hidden_layers=4)
model_orig = MambaForCausalLM_orig(config)

torch.manual_seed(34567)
config = MambaConfig(vocab_size=60, hidden_size=64, num_hidden_layers=4)
model = MambaForCausalLM(config)

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d
The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the mamba.py implementation for training. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [4]:
x = torch.randint(low=0, high=config.vocab_size-1, size=(16, 12))

y_orig = model_orig(x)
J_orig = y_orig.logits.sum()
J_orig.backward()

y = model(x)
J = y.logits.sum()
J.backward()

In [5]:
torch.allclose(y_orig.logits, y.logits, rtol=0.001)

True

In [6]:
gradients_same = True
for param1, param2 in zip(model_orig.parameters(), model.parameters()):
    if not torch.allclose(param1.grad, param2.grad, rtol=0.01):
        gradients_same = False
        break

print(gradients_same)

True
